In [ ]:
# In this demo, Oak Ridge has a set of training data while SLAC has testing data
# Oak Ridge will train and publish data in the catalog. 
# SLAC will retrieve and run it on their test data

In [25]:
%run 'onrl_setup.ipynb'

In [26]:
catalog_url = "http://127.0.0.1:5000/"
catalog = CatalogClient(catalog_url)

training_data = get_mnist_dataset(train=True)
oakridge_model = get_training_model(device="cpu")
train_model(oakridge_model, training_data)
serialized_model = get_serialized_model(oakridge_model)


Train Step: 0 [0/60000 (0%)]	Loss: 2.309363
Train Step: 1 [12800/60000 (21%)]	Loss: 2.281379
Train Step: 2 [25600/60000 (43%)]	Loss: 2.219163
Train Step: 3 [38400/60000 (64%)]	Loss: 1.920197
Train Step: 4 [51200/60000 (85%)]	Loss: 1.407151


In [27]:
def publish_oakridge(model_name, model):
    my_scope = "edu.onrl.slac_collab" # oakridge_domain
    my_type = "model"
    
    isModelFound, record_id, metadata = search_onrl_models(catalog, model_name)
    if isModelFound:
        time_dict = {'date_updated': datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}
        metadata.update(time_dict)
        return catalog.update_request(record_id, data_uri=None, scope=None, data_type=None, metadata=metadata, return_id=True)
    metadata = {"name":model_name, "date_created": datetime.now().strftime('%Y-%m-%dT%H:%M:%S'), 
                "date_updated": datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}
    return catalog.create_request(data_uri=model, scope=my_scope, data_type=my_type,metadata= metadata, return_id=True)
    

In [28]:
# Publish the trained model to the catalog 
MODEL_NAME="onrl_model_2_slac"
MODEL = serialized_model
published_id = publish_oakridge(MODEL_NAME, MODEL) 
print(f"Published model {MODEL_NAME} as {published_id}") 

Model found in the Catalog with the following metadata
{'date_created': '2023-02-14T16:13:07', 'date_updated': '2023-02-14T17:06:13', 'name': 'onrl_model_2_slac'}
Published model onrl_model_2_slac as 63ebf963bc06efbc3c6bdd65


In [30]:
# catalog.delete_request(published_id)
# catalog.purge()

True